In [1]:
!ls ../experiments/dim_multimodalmodel_hubert_xlarge_robertalarge_lazyload_lr1e5/results/dev.csv

../experiments/dim_multimodalmodel_hubert_xlarge_robertalarge_lazyload_lr1e5/results/dev.csv


In [2]:
import torch
def CCC_loss(pred, lab, m_lab=None, v_lab=None, is_numpy=True):
    """
    pred: (N, 3)
    lab: (N, 3)
    """
    if is_numpy:
        pred = torch.Tensor(pred).float()
        lab = torch.Tensor(lab).float()
    
    m_pred = torch.mean(pred, 0, keepdim=True)
    m_lab = torch.mean(lab, 0, keepdim=True)

    d_pred = pred - m_pred
    d_lab = lab - m_lab

    v_pred = torch.var(pred, 0, unbiased=False)
    v_lab = torch.var(lab, 0, unbiased=False)

    corr = torch.sum(d_pred * d_lab, 0) / (torch.sqrt(torch.sum(d_pred ** 2, 0)) * torch.sqrt(torch.sum(d_lab ** 2, 0)))

    s_pred = torch.std(pred, 0, unbiased=False)
    s_lab = torch.std(lab, 0, unbiased=False)

    ccc = (2*corr*s_pred*s_lab) / (v_pred + v_lab + (m_pred[0]-m_lab[0])**2)    
    return ccc

In [3]:
import pandas as pd
import numpy as np

In [4]:
preds = pd.read_csv('../experiments/dim_multimodalmodel_hubert_xlarge_robertalarge_lazyload_lr1e5/results/dev.csv')
preds.head()

,Filename,EmoAct_pred,EmoDom_pred,EmoVal_pred
0,MSP-PODCAST_0002_0033.wav,3.801689,3.490212,3.100219
1,MSP-PODCAST_0002_0039.wav,3.821425,3.659156,3.389325
2,MSP-PODCAST_0002_0051.wav,3.310054,3.430536,3.553863
3,MSP-PODCAST_0002_0059.wav,3.719033,3.617802,3.620624
4,MSP-PODCAST_0002_0061.wav,3.715548,3.668145,3.411520


In [5]:
df = pd.read_csv('/workspace/lucas.ueda/interspeech_ser/data/Labels/labels_consensus.csv')
df.head()

,FileName,EmoClass,EmoAct,EmoVal,EmoDom,SpkrID,Gender,Split_Set
0,MSP-PODCAST_0002_0033.wav,N,4.8,4.2,5.4,127,Female,Development
1,MSP-PODCAST_0002_0039.wav,N,4.0,4.2,4.2,127,Female,Development
2,MSP-PODCAST_0002_0051.wav,N,4.0,4.2,4.2,127,Female,Development
3,MSP-PODCAST_0002_0059.wav,X,4.0,3.8,4.0,128,Female,Development
4,MSP-PODCAST_0002_0061.wav,F,3.4,2.8,4.2,128,Female,Development


In [6]:
preds['FileName'] = preds['Filename']
preds = preds.merge(df[['FileName','EmoAct','EmoVal','EmoDom','Gender','SpkrID']], how='left', on = 'FileName')

In [7]:
# preds['EmoClass'].isnull().sum()

In [8]:
preds.head()

,Filename,EmoAct_pred,EmoDom_pred,EmoVal_pred,FileName,EmoAct,EmoVal,EmoDom,Gender,SpkrID
0,MSP-PODCAST_0002_0033.wav,3.801689,3.490212,3.100219,MSP-PODCAST_0002_0033.wav,4.8,4.2,5.4,Female,127
1,MSP-PODCAST_0002_0039.wav,3.821425,3.659156,3.389325,MSP-PODCAST_0002_0039.wav,4.0,4.2,4.2,Female,127
2,MSP-PODCAST_0002_0051.wav,3.310054,3.430536,3.553863,MSP-PODCAST_0002_0051.wav,4.0,4.2,4.2,Female,127
3,MSP-PODCAST_0002_0059.wav,3.719033,3.617802,3.620624,MSP-PODCAST_0002_0059.wav,4.0,3.8,4.0,Female,128
4,MSP-PODCAST_0002_0061.wav,3.715548,3.668145,3.411520,MSP-PODCAST_0002_0061.wav,3.4,2.8,4.2,Female,128


In [9]:
c = CCC_loss(preds[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds[['EmoAct','EmoVal','EmoDom']].values)

In [10]:
c[0].item(), c[1].item(), c[2].item()

(0.6283808946609497, 0.6678321361541748, 0.5473241209983826)

In [11]:
# Average F1-score using bootstrap
act = []
val = []
dom = []

for i in range(100):
    preds_subset = preds.sample(1000).reset_index(drop=True)
    c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)

    act.append(c[0].item())
    val.append(c[1].item())
    dom.append(c[2].item())

In [12]:
np.mean(act), np.mean(val), np.mean(dom)

(0.6294676977396011, 0.6668597739934922, 0.5492234995961189)

In [13]:
np.std(act), np.std(val), np.std(dom)

(0.019418184156193678, 0.017586698796048905, 0.02304713259407976)

In [14]:
np.min(act), np.min(val), np.min(dom)

(0.5690603852272034, 0.6131047010421753, 0.4880490005016327)

In [15]:
np.max(act), np.max(val), np.max(dom)

(0.6762366890907288, 0.7118237018585205, 0.5928717255592346)

# per gender

In [16]:
preds_subset = preds[preds['Gender'] == 'Female']
c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)
c

tensor([0.5296, 0.6459, 0.4560])

In [17]:
preds_subset = preds[preds['Gender'] == 'Male']
c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)
c

tensor([0.7030, 0.6480, 0.6656])